# 20th - Team Ice team Final Solution - Public LB 0.991, Private LB 0.992!

This notebook is the 20th place final submission for Team Ice team for Kaggle's IceCube - Neutrinos in Deep Ice competition. <b>Team members are junseonglee11 (@junseonglee11), Ayaan Jang(@ayaanjang).</b> This is an ensemble of the LSTM model of 6 (2 different versions).


* TFRecord Dataset Notebook: https://www.kaggle.com/code/junseonglee11/icecube-data-to-tfrecord-v2-1
* Train Notebook: https://www.kaggle.com/code/junseonglee11/20th-tensorflow-tfrecord-tpu-lstm-line-fit-train


# References
### Robin smith's: notebooks:
* https://www.kaggle.com/code/rsmits/tensorflow-lstm-model-inference
* https://www.kaggle.com/code/rsmits/tensorflow-lstm-model-training-tpu
* https://www.kaggle.com/code/rsmits/tensorflow-lstm-model-data-preprocessor/notebook 
I modified his notebook
* a. Converted the dataset to TFRecords
* b. Additional inputs and some preprocessing
* c. Changed the model (more RNN layers, GRU --> LSTM, GELU activation, rectified adam optimizer)

### Robert Hatch's notebook
* https://www.kaggle.com/code/roberthatch/lb-1-183-lightning-fast-baseline-with-polars
* It was crucial to improve our score. Used the results of this notebook as additional inputs in our model.

* |TFRecord dataset generation: https://www.kaggle.com/code/junseonglee11/icecube-data-to-tfrecord-v2-1

In this inference notebook I show that with the correct data pre-processing, model architecture and training setup an LSTM model can achieve the same performance as the current top performing Graph based models.

This notebook is based on the notebook originally published with LSTM training and inference: [3 LSTMs; with Data Picking and Shifting](https://www.kaggle.com/code/seungmoklee/3-lstms-with-data-picking-and-shifting). So if you like my notebooks don't forget the work that it is based!

This Inference notebook is for the largest part the same. It doesn't use the shifting as applied in the original work, it is based on only using 96 pulses and only 6 features.
It does ensemble 3 models but they are from the same training run (just different epochs).

Combining models from different model trainings with slightly different hyperparameters will very likely further increase the score. Increasing the number of bins and using more data for training is another way to increase the score of the models. Consider the Inference and Training notebooks a starting point to explore that yourself!

The training notebook for these models can be found [here](https://www.kaggle.com/code/rsmits/tensorflow-lstm-model-training-tpu). Note that I performed training on my local laptop (32GB RAM / NVidia 3070). For local training I loaded the files for 70 batches into RAM. The training notebook is equal to my local setup with the change that it loads multiple rounds of training data.

In the training notebook I will further explain what the differences are and how I improved the achieved score.

I hope you enjoy this notebook and if you do please give it an upvote :-)

!! Update in Latest Version: In the comments it was mentioned that using model files from a TPU training caused an error when trying to load the model. I've updated the load_model() with compile = False. This solves the error. Nothing else has changed in the notebook.

In [1]:
# Import Modules
import gc
import os
import multiprocessing
import time
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import tensorflow as tf
from tqdm import tqdm

In [2]:
# Directories and constants
home_dir = "/kaggle/input/icecube-neutrinos-in-deep-ice/"
test_format = home_dir + 'test/batch_{batch_id:d}.parquet'
   
# Model(s)
model_names = [
     # LB 0.993 (2) LSTM 7 layer
    "/kaggle/input/0419-lstm-1/epoch07-val_acc0.15238.h5",    
    "/kaggle/input/0419-lstm-1/epoch01-val_acc0.15396.h5",
    "/kaggle/input/0419-lstm-1/epoch02-val_acc0.15363.h5",
    "/kaggle/input/0419-lstm-1/epoch03-val_acc0.15471.h5",                        
    
    # LB 0.992 (2) LSTM 5 layer
    "/kaggle/input/0419-lstm-1/epoch03-val_acc0.15094.h5",
    "/kaggle/input/0419-lstm-1/epoch07-val_acc0.15104.h5",
    "/kaggle/input/0419-lstm-1/epoch08-val_acc0.15099.h5",     
    "/kaggle/input/0419-lstm-1/epoch05-val_acc0.15092.h5",                     
    
]

# Weight earned through CV
model_weights = np.array(
    [0.05836763, 0.72707093, 0.71994111, 0.78264521, 0.14015217, 0.71328186, 0.47582921, 0.99825985]                                                                                                                                                                                                           
)

#0 9 features time, pos diff added
#1 6 features pos --> pos - mean pos

input_norm_method = [
    2,    
    2,
    2,    
    2,
    
    2,
    2,
    2,
    2,
]

## Load Model(s)

In [3]:
# Load Models
models = []
feature_counts = []

for model_name in model_names:
    print(f'\n========== Model File: {model_name}')
    # Load Model
    model_path = model_name
    model = tf.keras.models.load_model(model_path, compile = False)
    models.append(model)      
    feature_count = model.inputs[0].shape[2]
    feature_counts.append(feature_count)
    # Model summary
    model.summary()
    
# Get Model Parameters
pulse_count = model.inputs[0].shape[1]




output_bins = model.layers[-1].weights[0].shape[-1]
bin_num = int(np.sqrt(output_bins))

# Model Parameter Summary
print("\n==== Model Parameters")
print(f"Bin Numbers: {bin_num}")
print(f"Maximum Pulse Count: {pulse_count}")
print(f"Features Count: {feature_count}")


========== Model File: /kaggle/input/0419-lstm-1/epoch07-val_acc0.15238.h5
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 9)]      0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 96, 9)        0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 96, 384)      310272      masking[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 96, 384)      886272      bidirectional[0][0]              
__________________

## Detector Information

In [4]:
# Load sensor_geometry
sensor_geometry_df = pd.read_csv(home_dir + "sensor_geometry.csv")

# Get Sensor Information
sensor_x = sensor_geometry_df.x
sensor_y = sensor_geometry_df.y
sensor_z = sensor_geometry_df.z

# Detector constants
c_const = 0.299792458  # speed of light [m/ns]

# Sensor Min / Max Coordinates
x_min = sensor_x.min()
x_max = sensor_x.max()
y_min = sensor_y.min()
y_max = sensor_y.max()
z_min = sensor_z.min()
z_max = sensor_z.max()

detector_length = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2 + (z_max - z_min)**2)
t_valid_length = detector_length / c_const

print(f"time valid length: {t_valid_length} ns")

time valid length: 6199.700247193777 ns


## Angle encoding edges

- It is efficient to train the model by classification task, initially.
- azimuth and zenith are independent
- azimuth distribution is flat and zenith distribution is sinusoidal.
  - Flat on the spherical surface
  - $\phi > \pi$ events are a little bit rarer than $\phi < \pi$ events, (maybe) because of the neutrino attenuation by earth.
- So, the uniform bin is used for azimuth, and $\left| \cos \right|$ bin is used for zenith

In [5]:
# Create Azimuth Edges
azimuth_edges = np.linspace(0, 2 * np.pi, bin_num + 1)
print(azimuth_edges)

# Create Zenith Edges
zenith_edges = []
zenith_edges.append(0)
for bin_idx in range(1, bin_num):
    zenith_edges.append(np.arccos(np.cos(zenith_edges[-1]) - 2 / (bin_num)))
zenith_edges.append(np.pi)
zenith_edges = np.array(zenith_edges)
print(zenith_edges)

[0.         0.19634954 0.39269908 0.58904862 0.78539816 0.9817477
 1.17809725 1.37444679 1.57079633 1.76714587 1.96349541 2.15984495
 2.35619449 2.55254403 2.74889357 2.94524311 3.14159265 3.33794219
 3.53429174 3.73064128 3.92699082 4.12334036 4.3196899  4.51603944
 4.71238898 4.90873852 5.10508806 5.3014376  5.49778714 5.69413668
 5.89048623 6.08683577 6.28318531]
[0.         0.3554212  0.50536051 0.62236849 0.72273425 0.81275556
 0.89566479 0.97338991 1.04719755 1.11797973 1.18639955 1.25297262
 1.31811607 1.38217994 1.4454685  1.50825556 1.57079633 1.63333709
 1.69612416 1.75941271 1.82347658 1.88862003 1.9551931  2.02361292
 2.0943951  2.16820274 2.24592786 2.32883709 2.41885841 2.51922417
 2.63623214 2.78617145 3.14159265]


## Define a function converts from prediction to angles

- Calculation of the mean-vector in a bin $\theta \in ( \theta_0, \theta_1 )$ and $\phi \in ( \phi_0, \phi_1 )$
  - $\vec{r} \left( \theta, ~ \phi \right) = \left< \sin \theta \cos \phi, ~ \sin \theta \sin \phi, ~ \cos \theta \right>$
  - $\bar{\vec{r}} = \frac{ \int_{\theta_{0}}^{\theta_{1}} \int_{\phi_0}^{\phi_1} \vec{r} \left( \theta, ~ \phi \right) \sin \theta \,d\phi \,d\theta }{ \int_{\theta_{0}}^{\theta_{1}} \int_{\phi_0}^{\phi_1} 1 \sin \theta \,d\phi \,d\theta }$
  - $ \int_{\theta_{0}}^{\theta_{1}} \int_{\phi_0}^{\phi_1} 1 \sin \theta \,d\phi \,d\theta = \left( \phi_1 - \phi_0 \right) \left( \cos \theta_0 - \cos \theta_1 \right)$
  - $
\int_{\theta_{0}}^{\theta_{1}} \int_{\phi_0}^{\phi_1} {r}_{x} \left( \theta, ~ \phi \right) \sin \theta \,d\phi \,d\theta = 
\int_{\theta_{0}}^{\theta_{1}} \int_{\phi_0}^{\phi_1} \sin^2 \theta \cos \phi \,d\phi \,d\theta = 
\left( \sin \phi_1 - \sin \phi_0 \right) \left( \frac{\theta_1 - \theta_0}{2} - \frac{\sin 2 \theta_1 - \sin 2 \theta_0}{4} \right)
$
  - $
\int_{\theta_{0}}^{\theta_{1}} \int_{\phi_0}^{\phi_1} {r}_{y} \left( \theta, ~ \phi \right) \sin \theta \,d\phi \,d\theta = 
\int_{\theta_{0}}^{\theta_{1}} \int_{\phi_0}^{\phi_1} \sin^2 \theta \sin \phi \,d\phi \,d\theta = 
\left( \cos \phi_0 - \cos \phi_1 \right) \left( \frac{\theta_1 - \theta_0}{2} - \frac{\sin 2 \theta_1 - \sin 2 \theta_0}{4} \right)
$
  - $
\int_{\theta_{0}}^{\theta_{1}} \int_{\phi_0}^{\phi_1} {r}_{z} \left( \theta, ~ \phi \right) \sin \theta \,d\phi \,d\theta = 
\int_{\theta_{0}}^{\theta_{1}} \int_{\phi_0}^{\phi_1} \sin \theta \cos \theta \,d\phi \,d\theta = 
\left( \phi_1 - \phi_0 \right) \left( \frac{\cos 2 \theta_0 - \cos 2 \theta_1}{4} \right)
$

In [6]:
angle_bin_zenith0 = np.tile(zenith_edges[:-1], bin_num)
angle_bin_zenith1 = np.tile(zenith_edges[1:], bin_num)
angle_bin_azimuth0 = np.repeat(azimuth_edges[:-1], bin_num)
angle_bin_azimuth1 = np.repeat(azimuth_edges[1:], bin_num)

angle_bin_area = (angle_bin_azimuth1 - angle_bin_azimuth0) * (np.cos(angle_bin_zenith0) - np.cos(angle_bin_zenith1))
angle_bin_vector_sum_x = (np.sin(angle_bin_azimuth1) - np.sin(angle_bin_azimuth0)) * ((angle_bin_zenith1 - angle_bin_zenith0) / 2 - (np.sin(2 * angle_bin_zenith1) - np.sin(2 * angle_bin_zenith0)) / 4)
angle_bin_vector_sum_y = (np.cos(angle_bin_azimuth0) - np.cos(angle_bin_azimuth1)) * ((angle_bin_zenith1 - angle_bin_zenith0) / 2 - (np.sin(2 * angle_bin_zenith1) - np.sin(2 * angle_bin_zenith0)) / 4)
angle_bin_vector_sum_z = (angle_bin_azimuth1 - angle_bin_azimuth0) * ((np.cos(2 * angle_bin_zenith0) - np.cos(2 * angle_bin_zenith1)) / 4)

angle_bin_vector_mean_x = angle_bin_vector_sum_x / angle_bin_area
angle_bin_vector_mean_y = angle_bin_vector_sum_y / angle_bin_area
angle_bin_vector_mean_z = angle_bin_vector_sum_z / angle_bin_area

angle_bin_vector = np.zeros((1, bin_num * bin_num, 3))
angle_bin_vector[:, :, 0] = angle_bin_vector_mean_x
angle_bin_vector[:, :, 1] = angle_bin_vector_mean_y
angle_bin_vector[:, :, 2] = angle_bin_vector_mean_z

angle_bin_vector_unit = angle_bin_vector[0].copy()
angle_bin_vector_unit /= np.sqrt((angle_bin_vector_unit**2).sum(axis=1).reshape((-1, 1)))

In [7]:
def pred_to_angle(_pred, power = 1.35, epsilon = 1e-8):
    # Convert prediction
    pred = _pred.copy()
    pred**= power    
    
    
    pred_vector = (pred.reshape((-1, bin_num**2, 1)) * angle_bin_vector).sum(axis = 1)
    
    # Normalize
    pred_vector_norm = np.sqrt((pred_vector**2).sum(axis = 1))
    mask = pred_vector_norm < epsilon
    pred_vector_norm[mask] = 1
    
    # Assign <1, 0, 0> to very small vectors (badly predicted)
    pred_vector /= pred_vector_norm.reshape((-1, 1))
    pred_vector[mask] = np.array([1., 0., 0.])
    
    # Convert to angle
    azimuth = np.arctan2(pred_vector[:, 1], pred_vector[:, 0])
    azimuth[azimuth < 0] += 2 * np.pi
    zenith = np.arccos(pred_vector[:, 2])
    
    # Mask bad norm predictions as 0, 0
    azimuth[mask] = 0.
    zenith[mask] = 0.
    
    return azimuth, zenith

## Weighted-Vector Ensemble

In [8]:
def weighted_vector_ensemble(angles, weight):
    # Convert angle to vector
    vec_models = list()
    for angle in angles:
        az, zen = angle
        sa = np.sin(az)
        ca = np.cos(az)
        sz = np.sin(zen)
        cz = np.cos(zen)
        vec = np.stack([sz * ca, sz * sa, cz], axis=1)
        vec_models.append(vec)
    vec_models = np.array(vec_models)

    # Weighted-mean
    vec_mean = (weight.reshape((-1, 1, 1)) * vec_models).sum(axis=0) / weight.sum()
    vec_mean /= np.sqrt((vec_mean**2).sum(axis=1)).reshape((-1, 1))

    # Convert vector to angle
    zenith = np.arccos(vec_mean[:, 2])
    azimuth = np.arctan2(vec_mean[:, 1], vec_mean[:, 0])
    azimuth[azimuth < 0] += 2 * np.pi
    
    return azimuth, zenith

## Single event reader function

- Pick-up important data points first
    - Rank 3 (First)
        - not aux, in valid time window
    - Rank 2
        - not aux, out of valid time window
    - Rank 1
        - aux, in valid time window
    - Rank 0 (Last)
        - aux, out of valid time window
    - In each ranks, take pulses from highest charge

In [9]:
# Placeholder
open_batch_dict = dict()

# Read single event from batch_meta_df
def read_event(event_idx, batch_meta_df, pulse_count):
    # Read metadata
    batch_id, first_pulse_index, last_pulse_index = batch_meta_df.iloc[event_idx][["batch_id", "first_pulse_index", "last_pulse_index"]].astype("int")

    # close past batch df
    if batch_id - 1 in open_batch_dict.keys():
        del open_batch_dict[batch_id - 1]

    # open current batch df
    if batch_id not in open_batch_dict.keys():
        open_batch_dict.update({batch_id: pd.read_parquet(test_format.format(batch_id=batch_id))})
    
    batch_df = open_batch_dict[batch_id]
    
    # Read event
    event_feature = batch_df[first_pulse_index:last_pulse_index + 1]
    sensor_id = event_feature.sensor_id
    
    # Merge features into single structured array
    dtype = [("time", "float16"),
             ("charge", "float16"),
             ("auxiliary", "float16"),
             ("x", "float16"),
             ("y", "float16"),
             ("z", "float16"),
             ("rank", "short")]    
    
    # Create event_x
    event_x = np.zeros(last_pulse_index - first_pulse_index + 1, dtype)
    event_x["time"] = event_feature.time.values - event_feature.time.min()
    event_x["charge"] = event_feature.charge.values
    event_x["auxiliary"] = event_feature.auxiliary.values
    event_x["x"] = sensor_geometry_df.x[sensor_id].values
    event_x["y"] = sensor_geometry_df.y[sensor_id].values
    event_x["z"] = sensor_geometry_df.z[sensor_id].values

    # For long event, pick-up
    if len(event_x) > pulse_count:
        # Find valid time window
        t_peak = event_x["time"][event_x["charge"].argmax()]
        t_valid_min = t_peak - t_valid_length
        t_valid_max = t_peak + t_valid_length
        t_valid = (event_x["time"] > t_valid_min) * (event_x["time"] < t_valid_max)

        # Rank
        event_x["rank"] = 2 * (1 - event_x["auxiliary"]) + (t_valid)

        # Sort by Rank and Charge (important goes to backward)
        event_x = np.sort(event_x, order = ["rank", "charge"])

        # pick-up from backward
        event_x = event_x[-pulse_count:]

        # Sort events by time 
        event_x = np.sort(event_x, order = "time")

    return event_idx, len(event_x), event_x

## Test metadata

In [10]:
# Read Test Meta data
test_meta_df = pq.read_table(home_dir + 'test_meta.parquet').to_pandas()
batch_counts = test_meta_df.batch_id.value_counts().sort_index()

batch_max_index = batch_counts.cumsum()
batch_max_index[test_meta_df.batch_id.min() - 1] = 0
batch_max_index = batch_max_index.sort_index()

# Support Function
def test_meta_df_spliter(batch_id):
    return test_meta_df.loc[batch_max_index[batch_id - 1]:batch_max_index[batch_id] - 1]

## Read test data and predict batchwise

# ⚡🧊⚡[LB 1.183] Polar Lightning
I took this from the invaluable works of roberthatch  
https://www.kaggle.com/code/roberthatch/lb-1-183-lightning-fast-baseline-with-polars/comments

In [11]:
## Configuration parameters
#MODE = 'train'
MODE = 'test'

# USE_POLARS = False
USE_POLARS = True

# TRAIN_MAX_EVENTS = 20000
TRAIN_MAX_EVENTS = None
TRAIN_BATCH_START = 1
TRAIN_N_BATCHES = 1

## I pulled in one piece of older code to demonstrate "before and after".
## Set to True (and USE_POLARS=False) if interested in seeing the difference.
USE_UNOPTIMIZED = False


#### HYPERPARAMETERS ####

## For setting auxiliary = False
## Hand-tuned and hand-validated, I mostly used batches 100-105, and probably early on also touched batch 1.
## TODO: revisit the deep core logic now that I've learned about the deep veto layer: https://www.kaggle.com/competitions/icecube-neutrinos-in-deep-ice/discussion/381702
FIND_BEST_POINTS = True
MIN_PRIMARY_DATAPOINTS = 2
MAX_Z = 3
MAX_DEEP_Z = 1
MAX_T = 350
MAX_DEEP_T = 180
if USE_POLARS:
    ## Only implmented in polars version.
    AUX_FALSE_WEIGHT = 0.01

## Ensemble and algorithm selection parameters.
## First weight is center of charge algorithm introduced in this notebook.
## Second weight is the unweighted version. Which turns out to be the least-squares algorithm: https://www.kaggle.com/competitions/icecube-neutrinos-in-deep-ice/discussion/381747
USE_ENSEMBLE = True
WEIGHTS = [0.58, 0.42]
if not USE_ENSEMBLE and not USE_POLARS:
    ALGORITHM = 'center of charge'
#     ALGORITHM = 'least squares'
    if ALGORITHM == 'least squares':
        USE_WEIGHTED_LEAST_SQUARES = False


## Constants
INPUT_DIR = '/kaggle/input/icecube-neutrinos-in-deep-ice'

## Basic configuration override logic
if MODE == 'test':
    TRAIN_MAX_EVENTS = None
if USE_ENSEMBLE:
    USE_WEIGHTED_LEAST_SQUARES = False
    
if USE_POLARS:
    try:
        import polars as pl
    except:
        print('Installing polars, please wait about 35 seconds...')
        !pip install /kaggle/input/polars01516/polars-0.15.16-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
        import polars as pl
        
import numpy as np
import pandas as pd
import math
import time
import gc
from tqdm.notebook import tqdm
tqdm.pandas()

## Condensed for space. See here for expanded original version: https://www.kaggle.com/code/sohier/mean-angular-error
def angular_dist_score(az_true, zen_true, az_pred, zen_pred):
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    return np.average(np.abs(np.arccos(scalar_prod)))

## TODO: It would be good to benchmark versus other implementations, like arctan2 used here: https://www.kaggle.com/code/shlomoron/icecube-eda-pca-baseline-cv-1-28-lb-1-274 

## This version has a small optimization trick, calculating azimuth without regard for z or zenith
## This version is suboptimal if the vectors are already unit vectors, or if you need 3d unit vectors again later for some other step.
def angles_from_vectors(vectors):
    v_squared = np.square(vectors)
    
    ## Shortcut optimization for azimuth: calculate 2d unit vectors for x and y independent of z
    xy_sq = np.sum(v_squared[:, 0:2], axis=1)
    xy_d = np.sqrt(xy_sq)[:, None]
    np.seterr(divide='ignore', invalid='ignore') ## Turn off the warning temporarily
    vectors[:, 0:2] = np.where(xy_d == 0, xy_d, vectors[:, 0:2]/xy_d)

    ## For z, use full 3d unit vector
    d = np.sqrt(xy_sq + v_squared[:, 2])
    vectors[:, 2] = np.where(d == 0, d, vectors[:, 2]/d)
    np.seterr(divide='warn', invalid='warn') ## Turn back on

    ## As mentioned by others, clip solely to avoid floating point errors, the unit vectors should already be within this range.
    vectors =  np.clip(vectors, -1, 1)

    azimuth = np.arccos(vectors[:, 0])
    ## if y < 0, convert from quadrants 1 and 2 to quadrants 3 and 4
    azimuth = np.where(vectors[:, 1] >= 0, azimuth, 2*math.pi - azimuth)
    azimuth = np.where(np.isfinite(azimuth), azimuth, 0.0)

    zenith = np.arccos(vectors[:, 2])
    ## IMPORTANT: zenith angles are not evenly distributed, so set the error case to pi/2!
    ## (even though x, y, z might be. It would be a fun exercise to check if random values
    ##  for x, y, z converted to zenith angles would match the observed distribution of zenith angles in the train labels)
    zenith = np.where(np.isfinite(zenith), zenith, math.pi/2)

    return np.stack([azimuth, zenith], axis=1)

## Takes a list of azimuth np arrays, a list of zenith np arrays, and an optional list of numerical weights,
## and ensembles into a final direction.
##
## It's not really optimal in terms of lines of code nor performance,
## since in most or all cases you are converting a unit vector to an angle,
## converting back to a unit vector, averaging, then converting to the final angle.
## However, it is quite convenient, because you can always use this at the end
## to ensemble the results originating from any number of notebooks or sources.
def average_angles(az_list, zen_list, weights=None):
    assert(len(az_list) == len(zen_list))
    total = az_list[0].shape[0]
    x = np.zeros(total)
    y = np.zeros(total)
    z = np.zeros(total)
    for i in range(len(az_list)):
        w = 1
        if weights is not None:
            w = weights[i]
        az = az_list[i]
        zen = zen_list[i]
        assert(az.shape[0] == total)
        assert(zen.shape[0] == total)
        if not (np.all(np.isfinite(az)) and
                np.all(np.isfinite(zen))):
            raise ValueError("All arguments must be finite")
        sz = np.sin(zen)
        x += w*np.cos(az)*sz
        y += w*np.sin(az)*sz
        z += w*np.cos(zen)
    tot_w = len(az_list)
    if weights is not None:
        tot_w = sum(weights)
    x = x / tot_w
    y = y / tot_w
    z = z / tot_w
    d = np.sqrt(np.square(x) + np.square(y) + np.square(z))
    x = x / d
    y = y / d
    z = z / d
    return angles_from_vectors(np.stack([x, y, z], axis=1))

def center_of_charge(batch):
    ## Groupby -> transform is the key trick to avoiding the dreaded 'for each event' loop,
    ## and thus getting ~30-60x speed boost improvement!
    ## If you need any min, max, mean or other simple value from the event group,
    ## you can precalculate it for each group and broadcast it
    batch['ev_t_min'] = batch.groupby('event_id')['time'].transform('min')
    batch['ev_t_max'] = batch.groupby('event_id')['time'].transform('max')
    
    ## Now we can just implement our formula! w0 and w1 are the time-weighted charge cases.
    ## Gather the values we need
    batch['w1'] = batch.charge * (batch.time - batch.ev_t_min) / (batch.ev_t_max - batch.ev_t_min)
    batch['w0'] = batch.charge - batch.w1
    batch['wx0'] = batch.x * batch.w0
    batch['wy0'] = batch.y * batch.w0
    batch['wz0'] = batch.z * batch.w0
    batch['wx1'] = batch.x * batch.w1
    batch['wy1'] = batch.y * batch.w1
    batch['wz1'] = batch.z * batch.w1
    df = batch[['w0', 'w1', 'wx0', 'wy0', 'wz0', 'wx1', 'wy1', 'wz1']]

    ## Calculate all the sums!
    df = df.groupby('event_id').sum()
    
    ## Now do the final divide of the weighted center by the sum of the weights.
    df[['wx0', 'wy0', 'wz0']] = df[['wx0', 'wy0', 'wz0']].div(df.w0, axis=0)
    df[['wx1', 'wy1', 'wz1']] = df[['wx1', 'wy1', 'wz1']].div(df.w1, axis=0)
    
    ## The direction the neutrino is traveling FROM is point0 - point1, instead of point1 - point0.
    ## Counter-intuitive to me, but fortunately, easy to notice and correct if your score is > 1.57 instead of less.
    df[['x', 'y', 'z']] = df[['wx0', 'wy0', 'wz0']].values - df[['wx1', 'wy1', 'wz1']].values

    df = df[['x', 'y', 'z']]
    df[['azimuth', 'zenith']] = angles_from_vectors(df.values)

    return(df[['azimuth', 'zenith']])

def least_squares(batch, weighted=False):
    batch['xt'] = batch.x * batch.time
    batch['yt'] = batch.y * batch.time
    batch['zt'] = batch.z * batch.time
    batch['tt'] = batch.time * batch.time
    if weighted:
        df = batch[['x', 'y', 'z', 'time', 'xt', 'yt', 'zt', 'tt']] * batch.charge.values[:, None]
        df['charge'] = batch.charge
        df = df.groupby('event_id').sum()
        df = df.div(df.charge, axis=0)
    else:
        df = batch[['x', 'y', 'z', 'time', 'xt', 'yt', 'zt', 'tt']]
        df = df.groupby('event_id').mean()
    df[['x', 'y', 'z']] = (
                              (df[['xt', 'yt', 'zt']].values - (df[['x', 'y', 'z']].values * df['time'].values[:, None]))
                            / (df['tt'].values - (df.time.values * df.time.values))[:, None]
                          )
    ## Reverse it
    df = -df[['x', 'y', 'z']]
    df[['azimuth', 'zenith']] = angles_from_vectors(df.values)
    return df[['azimuth', 'zenith']]


def process_batch(batch_id, sensor, max_events=None):
    print('load batch...')
    batch = pd.read_parquet(f'{INPUT_DIR}/{MODE}/batch_{batch_id}.parquet')

    ## Limit to max_events
    if max_events is not None:
        batch_i = batch.reset_index()
        event_ids = batch_i.event_id.drop_duplicates()
        end_index = event_ids.index[max_events]
        batch = batch_i[:end_index].set_index('event_id')
    print(batch.shape)

    ## Merge in sensor x,y,z data
    batch = batch.reset_index().merge(sensor, how='left', on='sensor_id', left_index=False).set_index('event_id')

    ## The logic for auxiliary = False is very basic, we can improve on it. Initial discussion here: 
    if FIND_BEST_POINTS:
        batch = find_best_points_pandas(batch)

    ## Limit to primary (aux=False) datapoints if there's enough of them.
    ## For event_ids with too few, set all rows to aux=False. This handles these cases without any for loop logic.
    ## MIN_PRIMARY_DATAPOINTS is a tuned value, based on optimizing the score on batches 101-110.
    ## But the result was 'as low as possible'.
    batch['primary_count'] = batch.groupby('event_id')['auxiliary'].transform('count') - batch.groupby('event_id')['auxiliary'].transform('sum')
    batch.loc[batch.primary_count < MIN_PRIMARY_DATAPOINTS, 'auxiliary'] = False
    batch = batch[batch.auxiliary == False]
    print(batch.shape)

    if USE_ENSEMBLE or ALGORITHM == 'center of charge':
        df = center_of_charge(batch)
        if USE_ENSEMBLE:
            df1 = df
    if USE_ENSEMBLE or ALGORITHM == 'least squares':
        df = least_squares(batch, weighted=USE_WEIGHTED_LEAST_SQUARES)
    if USE_ENSEMBLE:
        df[['azimuth', 'zenith']] = average_angles([df1.azimuth.values, df.azimuth.values],
                                                   [df1.zenith.values, df.zenith.values], 
                                                   weights=WEIGHTS)
    return df

def proximity(e, col):
    ## Since we explode the data to an NxN array, if N is too high it will take a long time and anyways we'll run out of memory.
    ## TODO: see how high we can go before we run out of memory
    ## TODO: consider subsampling instead of simply returning the baseline auxiliary setting?
    ##       And/or splitting on string_id to get much smaller groups
    if e.shape[0] > 2000:
        return e[:, col['auxiliary']]

    ## The magic None in the line below is a new thing I learned while working on this notebook.
    ## It is shorthand for np.newaxis, and broadcasts the array to another dimension.
    ## This allows us to create an NxN array for each event, so that we can
    ## check if *any* other row in the event meets our proximity in time and height requirements.
    ## For any matching pairs, then we set both rows as auxiliary = False. Rows without matches are auxiliary = True.
    deltas = np.abs(e[:, [col['string_id'], col['depth_id'], col['time']]] - e[:, None, [col['string_id'], col['depth_id'], col['time']]])
    dz = deltas[:, :, 1]

    ## if same depth or different string id, ignore by setting dz > the max threshold used later.
    dz[(dz == 0) | (deltas[:, :, 0] != 0)] = MAX_Z + MAX_DEEP_Z + 1

    ## if sensor is not a deep ice sensor, and time > MAX_T, ignore
    mask = (e[:, col['sensor_id']] < 4680)
    mask = np.broadcast_to(mask, (mask.shape[0], mask.shape[0])).T
    dz[mask & (deltas[:, :, 2] > MAX_T)] = MAX_Z + MAX_DEEP_Z + 1
    ## if sensor IS a deep ice sensor, and time > MAX_DEEP_T, ignore
    mask = (e[:, col['sensor_id']] >= 4680)
    mask = np.broadcast_to(mask, (mask.shape[0], mask.shape[0])).T
    dz[mask & (deltas[:, :, 2] > MAX_DEEP_T)] = MAX_Z + MAX_DEEP_Z + 1

    ## Now take the min (best) result for each row com
    dz = dz.min(axis=1)
    ## If no matches, the default, then everything is aux=True
    e[:, col['auxiliary']] = True
    ## If not deep ice and distance less than threshold, or deep ice and distance less than other threshold, then we have a match!
    e[((e[:, col['sensor_id']] < 4680) & (dz <= MAX_Z)) | ((e[:, col['sensor_id']] >= 4680) & (dz <= MAX_DEEP_Z)), col['auxiliary']] = False
    ## Return only the data needed to speed up the np.concatenate called next.
    return e[:, col['auxiliary']]

## You can ignore this one unless interested in a deep dive on performance optimization
## It is provided as a way of comparing the changes versus the pure numpy version.
## Comments removed from this copy to conserve vertical space.
def proximity_unoptimized(df):
    if df.shape[0] > 2000:
        return df
    deltas = np.abs(df[['string_id', 'depth_id', 'time']].values - df[['string_id', 'depth_id', 'time']].values[:, None, :])
    mask = (df.sensor_id < 4680)
    dz = deltas[:, :, 1]

    dz[(dz == 0) | (deltas[:, :, 0] != 0)] = MAX_Z + MAX_DEEP_Z + 1

    mask = (df.sensor_id < 4680)
    mask = np.broadcast_to(mask, (mask.shape[0], mask.shape[0])).T
    dz[mask & (deltas[:, :, 2] > MAX_T)] = MAX_Z + MAX_DEEP_Z + 1
    mask = (df.sensor_id >= 4680)
    mask = np.broadcast_to(mask, (mask.shape[0], mask.shape[0])).T
    dz[mask & (deltas[:, :, 2] > MAX_DEEP_T)] = MAX_Z + MAX_DEEP_Z + 1

    dz = dz.min(axis=1)
    df.auxiliary = True
    df.loc[((df.sensor_id < 4680) & (dz <= MAX_Z)) | ((df.sensor_id >= 4680) & (dz <= MAX_DEEP_Z)), 'auxiliary'] = False
    return df


def find_best_points_pandas(batch):
    if USE_UNOPTIMIZED:
        cols = ['sensor_id', 'time', 'auxiliary', 'string_id', 'depth_id']
        batch[cols] = batch[cols].groupby('event_id').progress_apply(proximity_unoptimized)
        return batch

    ## np.split used as a pure numpy equivalent of groupby
    ## Note this version didn't minimize the size of the inputs, but does make sure all dtypes are the same for an efficient np array.
    column_to_index = { k:v for v,k in enumerate(batch.columns)}
    events = np.split(batch.values.astype('float32'), np.unique(batch.index.values, return_index=True)[1][1:])

    ## Run each event sequentially in a list comprehension, then join back together with np.concatenate.
    ## So far tried and failed to find a reasonable solution to avoid this groupby > apply > join loop.
    ## Note that this line overrides the dtype of 'auxiliary' column to float32
    batch.auxiliary = np.concatenate([proximity(e, column_to_index) for e in tqdm(events)])
    return batch

def time_weighted_centering(batch, charge_weighted=True):
    ## Polars equivalent to groupby->transform is called 'over'. We again use this to get min and max without a for loop.
    batch = batch.with_columns([pl.col('time').min().over('event_id').alias('ev_t_min'),
                                pl.col('time').max().over('event_id').alias('ev_t_max')])
    if charge_weighted:
        batch = batch.with_columns((pl.col('charge') * (pl.col('time') - pl.col('ev_t_min'))
                                    / (pl.col('ev_t_max') - pl.col('ev_t_min'))).alias('w1'))
        batch = batch.with_columns((pl.col('charge') - pl.col('w1')).alias('w0'))
    else:
        batch = batch.with_columns(((pl.col('time') - pl.col('ev_t_min'))
                                    / (pl.col('ev_t_max') - pl.col('ev_t_min'))).alias('w1'))
        batch = batch.with_columns((pl.lit(1) - pl.col('w1')).alias('w0'))

    batch = batch.select(
        [
            pl.col('event_id'),
            pl.col('w0'),
            pl.col('w1'),
            (pl.col('x') * pl.col('w0')).alias('wx0'),
            (pl.col('y') * pl.col('w0')).alias('wy0'),
            (pl.col('z') * pl.col('w0')).alias('wz0'),
            (pl.col('x') * pl.col('w1')).alias('wx1'),
            (pl.col('y') * pl.col('w1')).alias('wy1'),
            (pl.col('z') * pl.col('w1')).alias('wz1'),
        ]
    ).collect().groupby('event_id', maintain_order=True).sum()

    ## The direction the neutrino is traveling FROM is point0 - point1, instead of point1 - point0.
    ## Counter-intuitive to me, but fortunately, easy to notice and correct if your score is > 1.57 instead of less.
    batch_values = batch.select(
        [
            ((pl.col('wx0') / pl.col('w0')) - (pl.col('wx1') / pl.col('w1'))).alias('x'),
            ((pl.col('wy0') / pl.col('w0')) - (pl.col('wy1') / pl.col('w1'))).alias('y'),
            ((pl.col('wz0') / pl.col('w0')) - (pl.col('wz1') / pl.col('w1'))).alias('z'),
        ]
    ).to_numpy()
    return angles_from_vectors(batch_values), batch


## We use the same numpy proximity function, the only difference is we convert from and to a Polars df instead of a Pandas df.
def find_best_points_polars(batch):
    ## Minimize the size of the inputs, and make sure all data types are the same for an efficient np array
    df = batch.select([pl.col('sensor_id'), pl.col('time'), pl.col('auxiliary'), pl.col('string_id'), pl.col('depth_id')])
    column_to_index = { k:v for v,k in enumerate(df.columns)}
    batch_values = df.to_numpy().astype('float32')

    ## Pure numpy equivalent of groupby
    events = np.split(batch_values, np.unique(batch.select(pl.col('event_id')), return_index=True)[1][1:])

    ## Run each event sequentially in a list comprehension, then join back together with np.concatenate. So far tried and failed to find a reasonable solution to avoid this groupby > apply > join loop.
    ## Rename 'auxiliary' -> 'best', and flip the boolean value
    batch = batch.with_columns(pl.Series(np.concatenate([proximity(e, column_to_index) for e in tqdm(events)]).astype('bool')).alias('best')).with_columns(pl.col('best').is_not())

    return batch



if USE_POLARS:
    sub = []
    #for batch_id in range(batch_id_start,batch_id_end):
def get_line_fit_angles(batch_id):
    print(MODE)
    ## Scan parquet is part of Polars lazy evaluation, so no cost yet,
    ## and it can figure out optimizations when we finally 'collect' it later.
    meta = pl.scan_parquet(f'{INPUT_DIR}/{MODE}_meta.parquet')

    print('load sensor data...')
    sensor = (pl.scan_csv(f'{INPUT_DIR}/sensor_geometry.csv')
                .with_columns([
                    pl.col('sensor_id').cast(pl.Int16),
                    (pl.col('sensor_id') // 60).alias('string_id'),
                    (pl.col('sensor_id') % 60).alias('depth_id'),                
                ])
             )

    print(sensor)

    if MODE == 'train':
        batch_id_start = TRAIN_BATCH_START
        batch_id_end = batch_id_start + TRAIN_N_BATCHES
    else:
        batch_id_start = meta.select(pl.col('batch_id')).collect()[0, 0]
        batch_id_end = meta.select(pl.col('batch_id')).collect()[-1, 0] + 1

    print(batch_id_start, batch_id_end)
    
    print(batch_id)
    t = time.time()
    max_events=TRAIN_MAX_EVENTS
    print('load batch...')
    batch = pl.scan_parquet(f'{INPUT_DIR}/{MODE}/batch_{batch_id}.parquet')

    ## Limit to max_events
    if max_events is not None:
        batch = batch.collect()
        last_event_id = batch.select(pl.col('event_id')).unique()[TRAIN_MAX_EVENTS-1, 0]
        batch = batch.lazy().filter(pl.col('event_id') <= last_event_id)

    ## Merge in sensor x,y,z data
    batch = batch.join(sensor, on='sensor_id', how='left').collect()

    ## The logic for auxiliary = False is very basic, we can improve on it. Initial discussion here: 
    if FIND_BEST_POINTS:
        batch = find_best_points_polars(batch)

    ## Use data point weights instead of filtering. Still set most points to 0 weight, unless they are the only data points available.
    ## MIN_PRIMARY_DATAPOINTS and AUX_FALSE_WEIGHT are tuned values, based on optimizing the score on batches 101-110.
    batch = batch.lazy().with_columns((pl.col('best').count().over('event_id') - pl.col('best').sum().over('event_id')).alias('best_count'))
    batch = batch.lazy().with_columns((pl.col('auxiliary').count().over('event_id') - pl.col('auxiliary').sum().over('event_id')).alias('non_aux_count'))
    batch = batch.with_columns(( pl.when( pl.col('best') | ((pl.col('best_count') < MIN_PRIMARY_DATAPOINTS) & (pl.col('non_aux_count') < MIN_PRIMARY_DATAPOINTS)) )
                                            .then(1.0)
                                            .otherwise(pl.when(pl.col('auxiliary').is_not())
                                                         .then(AUX_FALSE_WEIGHT)
                                                         .otherwise(0.0)
                                                    ) 
                                      ).alias('trust'))
    batch = batch.lazy().with_columns((pl.col('charge') * pl.col('trust')).alias('charge'))

    preds, events = time_weighted_centering(batch)
    if USE_ENSEMBLE:
        preds1 = preds
#             preds, events = time_weighted_centering(batch, charge_weighted=False)
        ## Instead of charge_weighted=False, set charge*aux to just equal aux.
        batch = batch.lazy().with_columns(pl.col('trust').alias('charge'))
        preds, events = time_weighted_centering(batch)
        preds = average_angles([preds1[:, 0], preds[:, 0]], [preds1[:, 1], preds[:, 1]], weights=WEIGHTS)

    '''
    if MODE == 'test':
        sub.append(events.select([pl.col('event_id'), pl.Series(preds[:, 0]).alias('azimuth'),
                                                     pl.Series(preds[:, 1]).alias('zenith')]))
    else:
        meta = meta.filter((pl.col('batch_id') >= batch_id_start) & (pl.col('batch_id') < batch_id_end))
        if isinstance(meta, pl.LazyFrame):
            meta = meta.collect()
        meta_values = meta.filter(pl.col('batch_id') == batch_id).select([pl.col('azimuth'), pl.col('zenith')]).to_numpy()
        if TRAIN_MAX_EVENTS is not None:
            print(angular_dist_score(meta_values[:TRAIN_MAX_EVENTS, 0], meta_values[:TRAIN_MAX_EVENTS, 1], preds[:, 0], preds[:, 1]))
        else:
            print(angular_dist_score(meta_values[:, 0], meta_values[:, 1], preds[:, 0], preds[:, 1]))
    '''
    print(f'Time: {time.time() - t:0.2f}s')
    return preds

#fitted_angle = get_line_fit_angles(1)


Installing polars, please wait about 35 seconds...
Processing /kaggle/input/polars01516/polars-0.15.16-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [12]:
def normalize_data(_data, option = 0, batch_id = 0):
    data = _data.copy()
    if(option==0 or option==2):
        data[:, :, 0] /= 1000   # time
        data[:, :, 1] /= 300    # charge
        data[:, :, 3:] /= 600   # space

        #time to diff_time
        data[:,:-1,0] = data[:,1:,0] - data[:,:-1,0]
        data[:,-1,0] = 0    

        #pseudo momentum (next time position - current position)
        pseudo_momentum = data[:, :, 3:6].copy()
        pseudo_momentum[:,:-1,:] = pseudo_momentum[:,1:,:] - data[:,:-1, 3:6]
        pseudo_momentum[:,-1,:] = 0

        for i in range(0, 3):
            pseudo_momentum[:,:-1,i][data[:,:-1,0]<0]=0

        data[:,:-1,0][data[:,:-1,0]<0] = 0

        data[:,:,6:] = pseudo_momentum.copy()
        del pseudo_momentum
        gc.collect()
        #data = np.append(data, pseudo_momentum, axis = 2)
        if(option==2):
            fitted_angle = get_line_fit_angles(batch_id)
            data = {'event_pulses': data, 'fitted_targets': fitted_angle}
    elif(option ==1):
        # Normalize - Version 6
        data[:, :, 0] /= 1000  # time
        data[:, :, 1] /= 300  # charge
        data[:, :, 3:] /= 600  # space

        # distance feature
        data[:, :, 6] = np.sqrt((data[:, :, 3] - data[:, :, 3].mean())**2 + (data[:, :, 4] - data[:, :, 4].mean())**2 + (data[:, :, 5] - data[:, :, 5].mean())**2)

        data[:, :, 3] = data[:, :, 3] - data[:, :, 3].mean()
        data[:, :, 4] = data[:, :, 4] - data[:, :, 4].mean()
        data[:, :, 5] = data[:, :, 5] - data[:, :, 5].mean()    

        # 충전과 거리 간의 상호작용 항 추가
        data[:, :, 6] = data[:, :, 1] * data[:, :, 6]
    return data

In [13]:
# Get Batch IDs
test_batch_ids = test_meta_df.batch_id.unique()

# Submission Placeholders
test_event_id = []
test_azimuth = []
test_zenith = []

# Batch Loop
for batch_id in test_batch_ids:
    # Batch Meta DF
    batch_meta_df = test_meta_df_spliter(batch_id)

    # Set Pulses
    test_x = np.zeros((len(batch_meta_df), pulse_count, 9), dtype = "float16")    
    test_x[:, :, 2] = -1    

    # Read Event Data
    def read_event_local(event_idx):
        return read_event(event_idx, batch_meta_df, pulse_count)
    
    # Multiprocess Events
    iterator = range(len(batch_meta_df))
    with multiprocessing.Pool() as pool:
        for event_idx, pulsecount, event_x in pool.map(read_event_local, iterator):
            # Features
            test_x[event_idx, :pulsecount, 0] = event_x["time"]
            test_x[event_idx, :pulsecount, 1] = event_x["charge"]
            test_x[event_idx, :pulsecount, 2] = event_x["auxiliary"]
            test_x[event_idx, :pulsecount, 3] = event_x["x"]
            test_x[event_idx, :pulsecount, 4] = event_x["y"]
            test_x[event_idx, :pulsecount, 5] = event_x["z"]
    
    del batch_meta_df
    gc.collect()
    

    '''
    test_x[:, :, 0] /= 1000  # time
    test_x[:, :, 1] /= 300  # charge
    test_x[:, :, 3:] /= 600  # space
        
    #230328 junseonglee11 time to time diff
    test_x[:,:-1,0] = test_x[:,1:,0] - test_x[:,:-1,0]
    test_x[:,-1,0] = 0    
    test_x[:,:-1,0][test_x[:,:-1,0]<0] = 0
    '''
    # Predict
    counter = 0
    pred_angles = []
    for model in models:
        # Normalize
        feature_count = feature_counts[counter]
        
        if(counter==0):
            test_tmp_x = normalize_data(test_x, input_norm_method[counter], batch_id)                                          
        elif(counter>0):
            if(input_norm_method[counter-1]!=input_norm_method[counter]):
                del test_tmp_x
                gc.collect()
                test_tmp_x = normalize_data(test_x, input_norm_method[counter], batch_id)     
                
                
        try:
            test_tmp_x = test_tmp_x[:,:,:feature_count]
        except:
            print('option 2 activated')
            
        
        pred_model = model.predict(test_tmp_x, verbose=0)
        if(input_norm_method[counter]==0 or input_norm_method[counter]==2):
            az_model, zen_model = pred_to_angle(pred_model['encoded_angle'])
        else:
            az_model, zen_model = pred_to_angle(pred_model)
        del pred_model
        gc.collect()
        pred_angles.append((az_model, zen_model))
        counter +=1
    
    # Get Predicted Azimuth and Zenith
    pred_azimuth, pred_zenith = weighted_vector_ensemble(pred_angles, model_weights)
    del pred_angles
    gc.collect()
    
    # Get Event IDs
    event_ids = test_meta_df.event_id[test_meta_df.batch_id == batch_id].values
    
    # Finalize 
    for event_id, azimuth, zenith in zip(event_ids, pred_azimuth, pred_zenith):
        if np.isfinite(azimuth) and np.isfinite(zenith):
            test_event_id.append(int(event_id))
            test_azimuth.append(azimuth)
            test_zenith.append(zenith)
        else:
            test_event_id.append(int(event_id))
            test_azimuth.append(0.)
            test_zenith.append(0.)
            
    del test_x, test_tmp_x, pred_azimuth, pred_zenith 
    gc.collect()

test
load sensor data...
naive plan: (run LazyFrame.describe_optimized_plan() to see the optimized plan)

   WITH_COLUMNS:
   [col("sensor_id").strict_cast(Int16), [(col("sensor_id")) // (60i32)].alias("string_id"), [(col("sensor_id")) % (60i32)].alias("depth_id")]
    CSV SCAN /kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv
    PROJECT */4 COLUMNS

661 662
661
load batch...


  0%|          | 0/3 [00:00<?, ?it/s]

Time: 0.05s
option 2 activated
option 2 activated
option 2 activated
option 2 activated
option 2 activated
option 2 activated
option 2 activated
option 2 activated


## Create Submission

In [14]:
# Create and Save Submission.csv
submission_df = pd.DataFrame({"event_id": test_event_id,
                              "azimuth": test_azimuth,
                              "zenith": test_zenith})
submission_df = submission_df.sort_values(by = ['event_id'])
submission_df.to_csv("submission.csv", index = False)

In [15]:
# Summary
submission_df.head()

,event_id,azimuth,zenith
0,2092,1.862817,1.432417
1,7344,3.329414,2.441870
2,9482,4.646359,1.568192
